In [1]:
import numpy as np
import gym
from go_ai import policies, game, metrics, data
from go_ai.models import value_model
import os
import random
import torch

# Hyperparameters

In [2]:
BOARD_SIZE = 4

In [3]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 256
NUM_EVAL_GAMES = 256

In [4]:
INIT_TEMP = 1
TEMP_DECAY = 3/4
MIN_TEMP = 1/64

In [5]:
BATCH_SIZE = 32

In [6]:
LOAD_SAVED_MODELS = False

# Data Parameters

In [7]:
EPISODES_DIR = 'episodes/'

In [8]:
CHECKPOINT_PATH = 'checkpoints/checkpoint_{}x{}.pt'.format(BOARD_SIZE, BOARD_SIZE)

In [9]:
DEMO_TRAJECTORY_PATH = 'logs/a_trajectory.png'

# Go Environment
Train on a small board for fast training and efficient debugging

In [10]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Model

In [11]:
curr_model = value_model.ValueNet(BOARD_SIZE)
checkpoint_model = value_model.ValueNet(BOARD_SIZE)

if LOAD_SAVED_MODELS:
    assert os.path.exists(CHECKPOINT_PATH)
    print("Starting from checkpoint")
else:
    torch.save(curr_model.state_dict(), CHECKPOINT_PATH)
    print("Initialized checkpoint") 

curr_model.load_state_dict(torch.load(CHECKPOINT_PATH))
checkpoint_model.load_state_dict(torch.load(CHECKPOINT_PATH))

curr_model

Initialized checkpoint


ValueNet(
  (convs): Sequential(
    (0): Conv2d(6, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(128, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
  )
  (fcs): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
  (criterion): BCEWithLogitsLoss()
)

# Policies

In [12]:
curr_policy = policies.QTempPolicy('Current', curr_model, INIT_TEMP)
checkpoint_policy = policies.QTempPolicy('Checkpoint', checkpoint_model, INIT_TEMP)

random_policy = policies.RandomPolicy()
greedy_policy = policies.QTempPolicy('Greedy', policies.greedy_val_func, temp=0)
human_policy = policies.HumanPolicy()

In [13]:
def decay_temps(policies, temp_decay, min_temp):
    for policy in policies:
        assert hasattr(policy, 'temp')
        policy.temp *= temp_decay
        if policy.temp < min_temp:
            policy.temp = min_temp
        print(f"{policy.name} temp decayed to {policy.temp}")

# Demo and Time Games

Symmetries

In [14]:
%%time
go_env.reset()
action = (1, 1)
next_state, _, _, _ = go_env.step(action)
metrics.plot_symmetries(next_state, 'logs/symmetries.jpg')

CPU times: user 335 ms, sys: 29.9 ms, total: 365 ms
Wall time: 181 ms


With replay memory

In [15]:
%%time
go_env.reset()
_,_ = game.pit(go_env, curr_policy, curr_policy, get_traj=True)

CPU times: user 569 ms, sys: 60.7 ms, total: 630 ms
Wall time: 255 ms


In [16]:
%%time
metrics.gen_traj_fig(go_env, curr_policy, DEMO_TRAJECTORY_PATH)

CPU times: user 3.46 s, sys: 187 ms, total: 3.65 s
Wall time: 3.28 s


# Train

In [ ]:
for iteration in range(ITERATIONS):
    print(f"Iteration {iteration}")
    
    # Make and write out the episode data
    _, replay_data = game.play_games(go_env, curr_policy, curr_policy, True, EPISODES_PER_ITERATION)
        
    # Process the data
    random.shuffle(replay_data)
    replay_data = data.replaylist_to_numpy(replay_data)

    # Optimize
    curr_model.optimize(replay_data, BATCH_SIZE)
    
    # Evaluate against checkpoint model and other baselines
    opp_winrate, _ = game.play_games(go_env, curr_policy, checkpoint_policy, False, EPISODES_PER_ITERATION)

    if opp_winrate > 0.6:
        # New parameters are significantly better. Accept it
        torch.save(curr_model.state_dict(), CHECKPOINT_PATH)
        checkpoint_model.load_state_dict(torch.load(CHECKPOINT_PATH))
        print(f"{100*opp_winrate:.1f}% Accepted new model")
        
        # Plot samples of states and response heatmaps
        metrics.gen_traj_fig(go_env, curr_policy, DEMO_TRAJECTORY_PATH)
        print("Plotted sample trajectory")
        
        rand_winrate, _ = game.play_games(go_env, curr_policy, random_policy, False, NUM_EVAL_GAMES)
        greed_winrate, _ = game.play_games(go_env, curr_policy, greedy_policy, False, NUM_EVAL_GAMES)

    elif opp_winrate >= 0.4:
        # Keep trying
        print(f"{100*opp_winrate:.1f}% Continuing to train current weights")
    else:
        # New parameters are significantly worse. Reject it.
        curr_model.load_state_dict(torch.load(CHECKPOINT_PATH))
        print(f"{100*opp_winrate:.1f}% Rejected new model")
        
    # Decay the temperatures if any
    decay_temps([curr_policy, checkpoint_policy], TEMP_DECAY, MIN_TEMP)

Current vs. Current:   1%|          | 1/128 [00:00<00:21,  5.90it/s, 0.0%]

Iteration 0


Current vs. Current: 100%|██████████| 128/128 [00:27<00:00,  4.66it/s, 59.8%]
Optimizing: 88it [00:01, 85.33it/s, 60.5%, 0.663L]
Current vs. Checkpoint: 100%|██████████| 128/128 [00:25<00:00,  4.93it/s, 60.9%]


60.9% Accepted new model


Current vs. Random:   0%|          | 0/128 [00:00<?, ?it/s]

...

Current vs. Random:   1%|          | 1/128 [00:00<00:12,  9.93it/s, 100.0%]

Plotted sample trajectory


Current vs. Current:   1%|          | 1/128 [00:00<00:23,  5.48it/s, 100.0%]

Current temp decayed to 0.015625
Checkpoint temp decayed to 0.015625
Iteration 31


Current vs. Current:  80%|███████▉  | 102/128 [00:22<00:05,  4.73it/s, 84.3%]

# Evaluate

Play against our AI

In [ ]:
set_temps([curr_policy, checkpoint_policy], 0)

In [ ]:
game.pit(go_env, human_policy, checkpoint_policy, False)